In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from ase.io import read, write
from ase.visualize import view
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
def main(filerec, index, voxl_size, axis, x_min, y_min,
         x_max, y_max, units, filename=None, _rotate=False):
    """
    Parameters:
    filerec: LabCore file record, containing 3D data
    index: index at which slice is made
    voxl_size: size of voxel in units
    axis: axis along which slice is made
    x_min: lower range of index in x direction
    y_min: lower range of index in y direction
    x_max: upper range of index in x direction
    y_max: upper range of index in y direction
    units: the units of voxl size
    filename: the png filename to store
    _rotate: rotate the data 180 to match exp directions
    """
    from matplotlib import pyplot as plt
    import numpy as np
    
    # data sanitisation
    index = int(index)
    voxl_size = list(map(float, voxl_size))
    if len(voxl_size) == 1:
        voxl_size *= 3
    axis = int(axis)
    x_min = int(x_min)
    y_min = int(y_min)
    
    if x_max == 'None':
        x_max = None
    elif x_max is not None:
        x_max = int(x_max)
    
    if y_max == 'None':
        y_max = None
    elif y_max is not None:
        y_max = int(y_max)
    
    # data = filerec['data']
    data = np.load(filerec)
    if _rotate:
        data = np.rot90(data, 2)
    if filename is None:
        filename = '.'.join(filerec.split('.')[:-1])
    a = tuple([slice(None) if i != axis else index for i in range(3)])
    img = data[a]
    a = tuple([slice(x_min, x_max), slice(y_min, y_max)])
    img = img[a]

    plt.imshow(img.T,
           #interpolation=None,
           origin='lower',
           cmap=plt.cm.Greys_r
          )
    ind = [1, 0, 0][axis]
    x = np.arange(0, img.shape[0] * voxl_size[ind], voxl_size[ind])
    nx = x.shape[0]
    n_labels = 5
    step_x = int(nx / (n_labels - 1))
    x_positions = np.arange(0 , nx, step_x)
    x_labels = x[::step_x]
    plt.xticks(x_positions, [f"{i:.2f}" for i in x_labels])
    plt.xlabel(f"{['Y', 'X', 'X'][axis]} {units}")
    
    ind = [2, 2, 1][axis]
    x = np.arange(0, img.shape[1] * voxl_size[ind], voxl_size[ind])
    nx = x.shape[0]
    n_labels = 5
    step_x = int(nx / (n_labels - 1))
    x_positions = np.arange(0 , nx, step_x)
    x_labels = x[::step_x]
    plt.yticks(x_positions, [f"{i:.2f}" for i in x_labels])
    plt.ylabel(f"{['Z', 'Z', 'Y'][axis]} {units}")
    
    plt.title(f"{['X', 'Y', 'Z'][axis]} at {index * voxl_size[axis]:.2f}{units}")
    if filename != 'dummy.png':
        plt.savefig(filename)

def main_ag(filerec, index, voxl_size, axis, x_min, y_min,
            x_max, y_max, units, centre_x=None, centre_y=None,
            length=1, theta=0, points=10, filename=None, _rotate=False,
            offset_x=0, offset_y=0):
    """
    Parameters:
    filerec: LabCore file record, containing 3D data
    index: index at which slice is made
    voxl_size: size of voxel in units
    axis: axis along which slice is made
    x_min: lower range of index in x direction
    y_min: lower range of index in y direction
    x_max: upper range of index in x direction
    y_max: upper range of index in y direction
    units: the units of voxl size
    centre: centre of cross hair
    length: diameter of cross hair
    theta: angle of cross hair
    points: number of points per cross hair
    filename: the png filename to store
    _rotate: rotate the data 180 to match exp directions
    """
    from matplotlib import pyplot as plt
    import numpy as np
    from scipy import interpolate
    solarised = False
    plt.rcParams.update({'font.size': 20,
                         'lines.linewidth': 3,
                         'figure.figsize': (8, 10),
                         'text.color': ('#657b83' if solarised else 'k'),
                         'axes.linewidth': 3,
                         'axes.labelcolor': ('#657b83' if solarised else 'k'),
                         'axes.edgecolor' : ('#586e75' if solarised else 'k'),
                         'xtick.color': ('#586e75' if solarised else 'k'),
                         'ytick.color': ('#586e75' if solarised else 'k'),
                         'xtick.major.width': 3,
                         'xtick.minor.width': 3,
                         'ytick.major.width': 3,
                         'ytick.minor.width': 3,
                         'legend.facecolor': ('#eee8d5' if solarised else 'inherit' ),
                         'savefig.bbox': 'tight',
                         'savefig.transparent': True})
    
    # data sanitisation
    index = int(index)
    voxl_size = list(map(float, voxl_size))
    if len(voxl_size) == 1:
        voxl_size *= 3
    axis = int(axis)
    x_min = int(x_min)
    y_min = int(y_min)
    
    if x_max == 'None':
        x_max = None
    elif x_max is not None:
        x_max = int(x_max)
    
    if y_max == 'None':
        y_max = None
    elif y_max is not None:
        y_max = int(y_max)
    
    # data = filerec['data']
    data = np.load(filerec)
    if _rotate:
        data = np.rot90(data, 2)
    if filename is None:
        filename = '.'.join(filerec.split('.')[:-1])
    a = tuple([slice(None) if i != axis else index for i in range(3)])
    img = data[a]
    a = tuple([slice(x_min, x_max), slice(y_min, y_max)])
    img = img[a].T
            
    if centre_x is None:
        centre_x = img.shape[0] / 2
    if centre_y is None:
        centre_y = img.shape[1] / 2
    
    centre = np.array([centre_x, centre_y])
        
    theta = theta * np.pi / 180
        
    x_pos = (np.array([np.cos(theta), np.sin(theta)])
             * length / 2 / voxl_size[0] + centre) + (np.array([np.cos(theta), np.sin(theta)])
             * offset_x / voxl_size[0])
    x_neg = (np.array([-np.cos(theta), -np.sin(theta)])
             * length / 2 / voxl_size[0] + centre) + (np.array([np.cos(theta), np.sin(theta)])
             * offset_x / voxl_size[0])
    y_pos = (np.array([-np.sin(theta), np.cos(theta)])
             * length / 2 / voxl_size[0] + centre) + (np.array([-np.sin(theta), np.cos(theta)])
             * offset_y / voxl_size[0])
    y_neg = (np.array([np.sin(theta), -np.cos(theta)])
             * length / 2 / voxl_size[0] + centre) + (np.array([-np.sin(theta), np.cos(theta)])
             * offset_y / voxl_size[0])
    
    fig, ax = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]})
    fig.tight_layout()

    ax[0].imshow(img,
           #interpolation=None,
           origin='lower',
           cmap=plt.cm.Greys_r
          )
    ax[0].plot([x_pos[0], x_neg[0]], [x_pos[1], x_neg[1]], 'b')
    ax[0].plot([y_pos[0], y_neg[0]], [y_pos[1], y_neg[1]], 'r')

    ind = [1, 0, 0][axis]
    x = np.arange(0, img.shape[0] * voxl_size[ind], voxl_size[ind])
    nx = x.shape[0]
    n_labels = 5
    step_x = int(nx / (n_labels - 1))
    x_positions = np.arange(0 , nx, step_x)
    x_labels = x[::step_x]
    # ax[0].set_xticks(x_positions, [f"{i:.2f}" for i in x_labels])
    ax[0].set_xlabel(f"{['Y', 'X', 'X'][axis]} {units}")

    ind = [2, 2, 1][axis]
    x = np.arange(0, img.shape[1] * voxl_size[ind], voxl_size[ind])
    nx = x.shape[0]
    n_labels = 5
    step_x = int(nx / (n_labels - 1))
    x_positions = np.arange(0 , nx, step_x)
    x_labels = x[::step_x]
    # ax[0].set_yticks(x_positions, [f"{i:.2f}" for i in x_labels])
    ax[0].set_ylabel(f"{['Z', 'Z', 'Y'][axis]} {units}")
    
    ax[0].set_title(f"{['X', 'Y', 'Z'][axis]} at {index * voxl_size[axis]:.2f}{units}")

    # second axis
    f = interpolate.interp2d(np.arange(img.shape[1]) * voxl_size[1], np.arange(img.shape[0]) * voxl_size[0], img, kind='cubic')
    x = np.linspace(x_neg[0], x_pos[0], points) * voxl_size[0]
    y = np.linspace(x_neg[1], x_pos[1], points) * voxl_size[0]
    z = f(x, y)
    ax[1].plot(np.linspace(-length / 2, length / 2, points), z.diagonal(), 'b')
    
    x = np.linspace(y_neg[0], y_pos[0], points) * voxl_size[0]
    y = np.linspace(y_neg[1], y_pos[1], points) * voxl_size[0]
    z = f(x, y)
    ax[1].plot(np.linspace(-length / 2, length / 2, points), z.diagonal(), 'r')

    ind = [1, 0, 0][axis]
    x = np.arange(0, img.shape[0] * voxl_size[ind], voxl_size[ind])
    nx = x.shape[0]
    n_labels = 5
    step_x = int(nx / (n_labels - 1))
    x_positions = np.arange(0 , nx, step_x)
    x_labels = x[::step_x]
    # ax[1].set_xticks(x_positions, [f"{i:.2f}" for i in x_labels])
    ax[1].set_xlabel(f"Axis {units}")
    
    ind = [2, 2, 1][axis]
    x = np.arange(0, img.shape[1] * voxl_size[ind], voxl_size[ind])
    nx = x.shape[0]
    n_labels = 5
    step_x = int(nx / (n_labels - 1))
    x_positions = np.arange(0 , nx, step_x)
    x_labels = x[::step_x]
    # ax[1].set_yticks(x_positions, [f"{i:.2f}" for i in x_labels])
    # ax[1].set_ylabel(f"{['X', 'Y', 'Z'][axis]} {units}")
    ax[1].set_ylabel(f"(Hz)")
    
    # ax[1].title(f"{['X', 'Y', 'Z'][axis]} at {index * voxl_size[axis]:.2f}{units}")

    if filename != 'dummy.png':
        plt.savefig(filename)

# Example

In [ ]:
example = read("example.vasp", format='vasp') 

In [ ]:
view(example)

In [ ]:
# view xy
main('example.npy', 44, [0.1], 2, 0, 0, None, None, 'A',
     'example_xy.png')

In [ ]:
# Y plotted against 42-1
main('example.npy', 58, [0.1], 0, 0, 44, None, -145, 'A',
     'example_yz.png')

In [ ]:
# X plotted against 0-10
main('example.npy', 119, [0.1], 1, 0, 44, None, -145, 'A',
     'example_xz.png')

In [ ]:
# view xy
@interact(filename='example.npy', z=(40, 95),
          centre_x=(40, 200), centre_y=(40, 160), length=5,
          points=200, offset_x=(-5, 5), offset_y=(-5, 5))
def interactive_xy(filename, z, centre_x, centre_y, length, points,
                   offset_x, offset_y):
    # save_filename = '.'.join(filename.split('.')[:-1]) + '.png'
    save_filename = 'dummy.png'
    main_ag(filename, z, [0.1], 2, 0, 0, None, None, '$\AA$',
            filename=save_filename, centre_x=centre_x, centre_y=centre_y,
            theta=0, length=length, points=points, _rotate=True, offset_x=offset_x,
            offset_y=offset_y)